<b>Uwaga:</b> Należy rozwiązać oba poniższe zadania. Na zaliczenie potrzeba co najmniej 3 pkt. 
<p></p>

1. <b>Zadanie 1.</b>(2x1.5 pkt): Zreplikować wyniki  związane z  materiałem na stronie <a href="https://thepythoncodingbook.com/2021/08/30/2d-fourier-transform-in-python-and-fourier-synthesis-of-images/">2d fourier transform in python and fourier synthesis of images</a>. Istotą tego zadania jest poprawne zrozumienie działania transformaty Fouriera na obrazach  - jego zaliczenie będzie polegało na odpowiedzi na dwa pytania teoretyczne.

2. <b>Zadanie 2.</b>(2x1 pkt): Napisać kod dwóch filtrów opartych o transformatę Fouriera: dolno i górnoprzepustowego. W pierwszym przypadku przykładowy obraz dobrej jakości zaburzyć kolejno trzema różnymi szumami dostępnymi w programie Gimp. W każdym przypadku zastosować filtr dolnoprzepustowy możliwie dokładnie przywracający obraz przed zaszumieniem. Następnie wygenerować filtr górnoprzepustowy wizualizujący w trzech powyższych przypadkach postać dodanego szumu.  Materiał referencyjny znajduje się m.in. na stronie
<a href="https://fairyonice.github.io/Low-and-High-pass-filtering-experiments.html">Low and High pass filtering experiments.html</a>. 